In [14]:

import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.49.0 to work with saurav_aml


In [15]:
from azureml.core import Dataset
from azureml.data.datapath import DataPath

default_ds = ws.get_default_datastore()

if 'insurance dataset' not in ws.datasets:
    Dataset.File.upload_directory(src_dir='data',
                              target=DataPath(default_ds, 'insurance-data/')
                              )

    #Create a tabular dataset from the path on the datastore (this may take a short while)
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'insurance-data/*.csv'))

    # Register the tabular dataset
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name='insurance dataset',
                                description='insurance data',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')

Dataset already registered.


In [16]:
import os
# Create a folder for the pipeline step files
experiment_folder = 'insurance_pipeline'
os.makedirs(experiment_folder, exist_ok=True)

print(experiment_folder)

insurance_pipeline


In [17]:
%%writefile $experiment_folder/prep_insurance.py
# Import libraries
import os
import argparse
import pandas as pd
import numpy as np
from azureml.core import Run
from sklearn.preprocessing import MinMaxScaler

# Get parameters
parser = argparse.ArgumentParser()
parser.add_argument("--input-data", type=str, dest='raw_dataset_id', help='raw dataset')
parser.add_argument('--prepped-data', type=str, dest='prepped_data', default='prepped_data', help='Folder for results')
args = parser.parse_args()
prep_folder = args.prepped_data

# Get the experiment run context
run = Run.get_context()

# load the data (passed as an input dataset)
print("Loading Data...")
df = run.input_datasets['raw_data'].to_pandas_dataframe()

# Log raw row count
row_count = (len(df))
run.log('raw_rows', row_count)


def Preprocessing(df):
    """Data Pre-processing"""
    # if '?' in the datset which we have to remove by NaN Values
    df = df.replace('?',np.NaN)

    # missing value treatment using fillna

    # we will replace the '?' by the most common collision type as we are unaware of the type.
    df['collision_type'].fillna(df['collision_type'].mode()[0], inplace = True)

    # It may be the case that there are no responses for property damage then we might take it as No property damage.
    df['property_damage'].fillna('NO', inplace = True)

    # again, if there are no responses fpr police report available then we might take it as No report available
    df['police_report_available'].fillna('NO', inplace = True)

    # let's extrat days, month and year from policy bind date
    df['policy_bind_date'] = pd.to_datetime(df['policy_bind_date'], errors = 'coerce')

    # let's encode the fraud report to numerical values
    df['fraud_reported'] = df['fraud_reported'].replace(('Y','N'),(0,1))

    # dropping unimportant columns
    df = df.drop(columns = [
        'policy_number', 
        'insured_zip', 
        'policy_bind_date', 
        'incident_date', 
        'incident_location', 
        '_c39', 
        'auto_year', 
        'incident_hour_of_the_day'])

    numeric_data = df._get_numeric_data()
    cat_data = df.select_dtypes(include=['object'])

    # Normalize the numeric columns
    scaler = MinMaxScaler()

    num_data_clean = scaler.fit_transform(numeric_data)

    clean_data = pd.concat([pd.get_dummies(cat_data), numeric_data], axis=1)

    return clean_data

dataPrep = Preprocessing(df)

# remove nulls
# insurance = insurance.dropna()

# # Normalize the numeric columns
# scaler = MinMaxScaler()
# num_cols = ['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree']
# diabetes[num_cols] = scaler.fit_transform(diabetes[num_cols])

# # Log processed rows
row_count = (len(df))
run.log('processed_rows', row_count)

# Save the prepped data
print("Saving Data...")
os.makedirs(prep_folder, exist_ok=True)
save_path_1 = os.path.join(prep_folder,'prep_data.csv')
dataPrep.to_csv(save_path_1, index=False, header=True)

# End the run
run.complete()

Overwriting insurance_pipeline/prep_insurance.py


In [18]:
%%writefile $experiment_folder/train_insurance.py
# Import libraries
from azureml.core import Run, Model
import argparse
import pandas as pd
import numpy as np
import joblib
import os
# from azureml.core import Run
import argparse, joblib, os
import argparse
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, classification_report, cohen_kappa_score
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import train_test_split
# from sklearn.tree import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

# Get parameters
parser = argparse.ArgumentParser()
parser.add_argument("--training-data", type=str, dest='training_data', help='training data')
args = parser.parse_args()
training_data = args.training_data

# Get the experiment run context
run = Run.get_context()

# load the prepared data file in the training folder
print("Loading Data...")
file_path = os.path.join(training_data,'prep_data.csv')
data_prep = pd.read_csv(file_path)


# Get parameters
import argparse
parser = argparse.ArgumentParser()
parser.add_argument("--n_estimators", type=int)
parser.add_argument("--min_samples_leaf", type=int)
parser.add_argument("--datafolder", type=str)

args, unknown = parser.parse_known_args()

ne = args.n_estimators
msl = args.min_samples_leaf

print(ne, msl)

X = data_prep.iloc[:, 0:-1]

y = data_prep.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=1234)

# Baseline Random forest based Model
rfc = RandomForestClassifier(n_estimators=ne, min_samples_leaf=msl)
rfcg = rfc.fit(X_train, y_train) # fit on training data
Y_predict = rfcg.predict(X_test)

# Get the probability score - Scored Probabilities
Y_prob = rfcg.predict_proba(X_test)[:, 1]

# Get Confusion matrix and the accuracy/score - Evaluate

cm    = confusion_matrix(y_test, Y_predict)
accuracy = accuracy_score(y_test, Y_predict)

# Create the confusion matrix dictionary
cm_dict = {"schema_type": "confusion_matrix",
           "schema_version": "v1",
           "data": {"class_labels": ["N", "Y"],
                    "matrix": cm.tolist()}
           }

run.log("TotalObservations", len(dataPrep))
run.log_confusion_matrix("ConfusionMatrix", cm_dict)
run.log("Accuracy", accuracy)

# # Save the model in the run outputs
# os.makedirs('outputs', exist_ok=True)
# joblib.dump(value=rfc, filename='outputs/insurance_model.pkl')

# # Complete the run
# run.complete()

# Save the trained model in the outputs folder
print("Saving model...")
os.makedirs('outputs', exist_ok=True)
model_file = os.path.join('outputs', 'insurance_model.pkl')
joblib.dump(value=rfc, filename=model_file)

# Register the model
print('Registering model...')
Model.register(workspace=run.experiment.workspace,
               model_path = model_file,
               model_name = 'diabetes_model',
               tags={'Training context':'Pipeline'},
               properties={'Accuracy': np.float(accuracy)})


run.complete()


Overwriting insurance_pipeline/train_insurance.py


In [19]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "saurav-compute-cluster"

try:
    # Check for existing compute target
    pipeline_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        pipeline_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        pipeline_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)
    

Found existing cluster, use it.


In [20]:
%%writefile $experiment_folder/experiment_env.yml
name: experiment_env
dependencies:
- python=3.8
- scikit-learn
- ipykernel
- matplotlib
- pandas
- pip
- pip:
  - azureml-defaults
  - pyarrow

Overwriting insurance_pipeline/experiment_env.yml


In [21]:

from azureml.core import Environment
from azureml.core.runconfig import RunConfiguration

# Create a Python environment for the experiment (from a .yml file)
experiment_env = Environment.from_conda_specification("experiment_env", experiment_folder + "/experiment_env.yml")

# Register the environment 
experiment_env.register(workspace=ws)
registered_env = Environment.get(ws, 'experiment_env')

# Create a new runconfig object for the pipeline
pipeline_run_config = RunConfiguration()

# Use the compute you created above. 
pipeline_run_config.target = pipeline_cluster

# Assign the environment to the run configuration
pipeline_run_config.environment = registered_env

print ("Run configuration created.")

Run configuration created.


In [25]:
from azureml.data import OutputFileDatasetConfig
from azureml.pipeline.steps import PythonScriptStep

# Get the training dataset
diabetes_ds = ws.datasets.get("insurance dataset")

# Create an OutputFileDatasetConfig (temporary Data Reference) for data passed from step 1 to step 2
prepped_data = OutputFileDatasetConfig("prepped_data")

# Step 1, Run the data prep script
prep_step = PythonScriptStep(name = "Prepare Data",
                                source_directory = experiment_folder,
                                script_name = "prep_insurance.py",
                                arguments = ['--input-data', diabetes_ds.as_named_input('raw_data'),
                                             '--prepped-data', prepped_data],
                                compute_target = pipeline_cluster,
                                runconfig = pipeline_run_config,
                                allow_reuse = True)

# Step 2, run the training script
train_step = PythonScriptStep(name = "Train and Register Model",
                                source_directory = experiment_folder,
                                script_name = "train_insurance.py",
                                arguments = ['--training-data', prepped_data.as_input()],
                                compute_target = pipeline_cluster,
                                runconfig = pipeline_run_config,
                                allow_reuse = True)

print("Pipeline steps defined")

Pipeline steps defined


In [27]:
from azureml.core import Experiment
from azureml.pipeline.core import Pipeline
from azureml.widgets import RunDetails

# Construct the pipeline
pipeline_steps = [prep_step, train_step]
pipeline = Pipeline(workspace=ws, steps=pipeline_steps)
print("Pipeline is built.")

# Create an experiment and run the pipeline
experiment = Experiment(workspace=ws, name = 'saurav-insurance-pipeline')
pipeline_run = experiment.submit(pipeline, regenerate_outputs=True)
print("Pipeline submitted for execution.")
RunDetails(pipeline_run).show()
pipeline_run.wait_for_completion(show_output=True)
     

Pipeline is built.
Created step Prepare Data [9a5816dd][eebeb9bf-8a77-42c6-ac1f-605f574a0f8f], (This step will run and generate new outputs)
Created step Train and Register Model [c978cd95][ee975cc2-3164-4089-a8c4-b31874ea0da8], (This step will run and generate new outputs)
Submitted PipelineRun 8015f779-539b-49d5-9e4d-9661d7cc8cdf
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/8015f779-539b-49d5-9e4d-9661d7cc8cdf?wsid=/subscriptions/7c248226-48dc-4d36-baa0-0f0883669328/resourcegroups/saurv_01/workspaces/saurav_aml&tid=0563aea9-b886-4b62-9457-a85924a13bd1
Pipeline submitted for execution.


_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

PipelineRunId: 8015f779-539b-49d5-9e4d-9661d7cc8cdf
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/8015f779-539b-49d5-9e4d-9661d7cc8cdf?wsid=/subscriptions/7c248226-48dc-4d36-baa0-0f0883669328/resourcegroups/saurv_01/workspaces/saurav_aml&tid=0563aea9-b886-4b62-9457-a85924a13bd1
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: fdd667c7-b56d-40c1-9d27-d3dfc55dee9f
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/fdd667c7-b56d-40c1-9d27-d3dfc55dee9f?wsid=/subscriptions/7c248226-48dc-4d36-baa0-0f0883669328/resourcegroups/saurv_01/workspaces/saurav_aml&tid=0563aea9-b886-4b62-9457-a85924a13bd1
StepRun( Prepare Data ) Status: Queued
StepRun( Prepare Data ) Status: Running

StepRun(Prepare Data) Execution Summary
StepRun( Prepare Data ) Status: Finished
{'runId': 'fdd667c7-b56d-40c1-9d27-d3dfc55dee9f', 'target': 'saurav-compute-cluster', 'status': 'Completed', 'startTimeUtc': '2023-03-22T17:43:10.182514Z', 'endTimeUtc': '2023-03-22T17:4

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "Execution failed. User process '/azureml-envs/azureml_f102ca25577426a3b1fe82a38ab699fe/bin/python' exited with status code 1. Please check log file 'user_logs/std_log.txt' for error details. Error: Traceback (most recent call last):\n  File \"train_insurance.py\", line 17, in <module>\n    from sklearn.tree import RandomForestClassifier\nImportError: cannot import name 'RandomForestClassifier' from 'sklearn.tree' (/azureml-envs/azureml_f102ca25577426a3b1fe82a38ab699fe/lib/python3.8/site-packages/sklearn/tree/__init__.py)\n\n",
        "messageParameters": {},
        "details": []
    },
    "time": "0001-01-01T00:00:00.000Z",
    "componentName": "CommonRuntime"
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Activity Failed:\n{\n    \"error\": {\n        \"code\": \"UserError\",\n        \"message\": \"Execution fai